In [1]:
import cv2   
import pandas as pd

face_model = cv2.face.LBPHFaceRecognizer_create()
face_model.read('facemodel.xml')
scale = 0.5  
feature_size = (96, 96) 

label_file = 'labels.csv'
df = pd.read_csv(label_file)
y_label = df.name

casc_file = "haarcascade_frontalface_default.xml"
frontal_face = cv2.CascadeClassifier(casc_file)  # Create the haar cascade

faces = []

cap = cv2.VideoCapture(0) # cam
cap.set(3, 1024) 
cap.set(4, 768) 

if (cap.isOpened()== False):  
    print("Could not open a camera") 


def detect_faces(image):
    bBoxes = frontal_face.detectMultiScale(image, scaleFactor=1.3, 
        minNeighbors=5, minSize=(30, 30)) 
    return bBoxes    
    
frame_counter = 0
while(cap.isOpened()): 
    ret, frame = cap.read() 

    if ret == True: 
        color_image = frame
        frame_counter += 1

        if frame_counter >= cap.get(cv2.CAP_PROP_FRAME_COUNT):
            frame_counter = 0 
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            
        color_image = cv2.resize(color_image, None,fx=scale,fy=scale,
                                 interpolation=cv2.INTER_AREA)

        if (frame_counter % 5) == 0:

            gray_frame = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)


            bBoxes = detect_faces(gray_frame)
            for bBox in bBoxes:
                (p,q,r,s)= bBox
                
                cv2.rectangle(color_image, (p,q), (p+r,q+s), (0,255,0), 2)


                crop_image = gray_frame[q:q+s, p:p+r]

                crop_image = cv2.resize(crop_image, feature_size) # ksb
                [pred_label, pred_conf]= face_model.predict(crop_image)

                box_bg = (0, 255, 0)
                cv2.rectangle(color_image, (p,q), (p+95,q-22), box_bg, cv2.FILLED)

                box_text= y_label[pred_label][:7]
                txt_color = (100,0,215)
                cv2.putText(color_image, box_text, (p+4, q-4), 
                            cv2.FONT_HERSHEY_PLAIN, 1.0, txt_color, 2)

            cv2.imshow('Mywindow', color_image) 

        if cv2.waitKey(10) & 0xFF == 27: 
            break
   
    else:  
        break

        
cap.release() 
cv2.destroyAllWindows()

AttributeError: module 'cv2' has no attribute 'face'

In [ ]:
pip install opencv-python